In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
# ha-ha classics

import numpy as np 

import pandas as pd

import matplotlib.pyplot as plt

import sklearn

import os 

# sklearn

from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import precision_recall_curve, roc_curve

# 2nd category

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier as lgbm

# 3rd category

import torch

import torch.nn as nn
import torch.optim as optim

# misc
from tqdm import tqdm

### Additional preprocessing

In [2]:
X = pd.read_csv('data/preprocessed_train_values.csv')
X_check = pd.read_csv('data/preprocessed_test_values.csv')
y = pd.read_csv('data/train_labels.csv')

del X['Unnamed: 0']
del X_check['Unnamed: 0']

In [3]:
X.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w,building_id
0,6,487,12198,2,30,6,5,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,802906
1,8,900,2812,2,10,8,7,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28830
2,21,363,8973,2,10,5,5,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,94947
3,22,418,10694,2,10,6,5,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,590882
4,11,131,1488,3,30,8,9,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,201944


In [197]:
columns_to_scale = X.columns[:7]
mask = []
for cols in columns_to_scale:
    mask.append(cols)

enc = StandardScaler()

X_to_scale = X[mask]

X.drop(mask,axis = 1)

X_encoded = enc.fit_transform(X_to_scale)

X[columns_to_scale] = X_encoded


del X_to_scale
del X_encoded

In [198]:
columns_to_scale = X_check.columns[:7]
mask = []
for cols in columns_to_scale:
    mask.append(cols)

enc = StandardScaler()

X_check_to_scale = X_check[mask]

X_check.drop(mask,axis = 1)

X_encoded = enc.fit_transform(X_check_to_scale)

X_check[columns_to_scale] = X_encoded


del X_check_to_scale
del X_encoded

### Attempts of feature engineering

The idea is to cluster the values by their geographical proximity, as it is likely that the level of damage dealt is quite equal in a geographical region due to similar buildings quality. As first three features are connected with the question, but obviously have different weights as impacts on the geographical placing, we will assign naive weights of e2,e1,1 accordingly and use KMeans to figure out at least the number of clusters. Before we found the amount of sensible clusters as 17 using Naive DBSCAN on test_data with eps = 35.


The epsilon hyperparameter was chosen thus to create sensible finite number of clusters. Clustering should be stable on all datasets to be considered as sensible.

In [199]:
from sklearn.cluster import KMeans

def create_clustering_data(dataset): #instance of np.ndarray
    subdata = dataset[:,0:3].copy()
    return subdata

def perform_clustering(subdata,col_weights = [100,10,1]):
    for i in range(len(col_weights)):
        subdata[:,i] *= col_weights[i]
        
    print(subdata.shape)
    #print('...clustering in process...')
    clustering = KMeans(17).fit(subdata)
    print('...clustering done.')
    #print(clustering.labels_.shape)
    
    return clustering.labels_


In [200]:
enc_cols = ['cluster_' + str(i) for i in range(17)]
print(enc_cols)

def append_clusters_to_dataset(main_set, clustering):
    enc = OneHotEncoder()
    ohe = enc.fit_transform(clustering.reshape(-1,1))
    ohe = ohe.toarray()
    add_on = pd.DataFrame(ohe,columns = enc_cols)
    
    X = pd.concat([main_set,add_on],axis = 1)
    return X

['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9', 'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16']


#### Processing full_train

In [183]:
sub_X = create_clustering_data(np.array(X))
clustering_main = perform_clustering(sub_X)

(260601, 3)
...clustering done.


In [186]:
X = append_clusters_to_dataset(X, clustering_main)
X.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land

#### Processing check

In [187]:
sub_X_check = create_clustering_data(np.array(X_check))
clustering_check = perform_clustering(sub_X_check)

(86868, 3)
...clustering done.


In [188]:
X_check = append_clusters_to_dataset(X_check, clustering_main)
X_check.shape

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land

### Train-test split

In [189]:
X_train,X_test,y_train,y_test = train_test_split(X.sort_values('building_id'),
                                                 y.sort_values('building_id')['damage_grade'],
                                                 test_size = 0.2,
                                                 random_state = 42)


In [190]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)

### Linear Models attempts

#### Training LogReg

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(penalty='l2')

param_grid = {'C' : np.logspace(-2,4,7)}

log_grid = GridSearchCV(model,param_grid=param_grid,scoring = 'accuracy',verbose=10,n_jobs=3)

log_grid.fit(X_train,y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    3.1s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    7.1s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    9.2s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:   15.3s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   19.5s
[Parallel(n_jobs=3)]: Done  35 out of  35 | elapsed:   25.9s finished


GridSearchCV(estimator=LogisticRegression(), n_jobs=3,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04])},
             scoring='accuracy', verbose=10)

#### Predicting on test_data

In [10]:
y_logreg_pred = log_grid.predict(X_test)

print(f1_score(y_logreg_pred,y_test,average = 'micro'))

0.5688877803572456


### Boosting

In [11]:
model = XGBClassifier()

param_grid = {'max_depth' : range(2,16,2),
             'num_estimators' : [10,20,50,100,200]}

boost_grid = GridSearchCV(model,param_grid=param_grid,scoring = 'accuracy',verbose=10,n_jobs=3)

print('--- / fitting / ---')

boost_grid.fit(X_train,y_train)



--- / fitting / ---
Fitting 5 folds for each of 35 candidates, totalling 175 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   56.3s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  3.0min
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:  4.0min
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  6.8min
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  9.4min
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed: 14.2min
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 19.0min
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed: 26.2min
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed: 34.7min
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed: 46.3min
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed: 58.9min
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 76.7min
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 95.9min
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 122.0min
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 150.4min
[Paralle

[06:33:01] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { num_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameter

In [13]:
y_boost_pred = boost_grid.predict(X_test)

print('='*100)
print('SCORE:')
print(f1_score(y_test,y_boost_pred,average='micro'))

print('='*100)

SCORE:
0.7360181117016174


The best model was (100,12)

In [216]:
### Additional re-check of non-val score with features added. Slight increase.

best_model = XGBClassifier(n_estimators = 100,max_depth = 12)

best_model.fit(X_train,y_train)

y_boost = best_model.predict(X_test)

print('='*100)
print('SCORE:')
print(f1_score(y_test,y_boost,average='micro'))

print('='*100)

SCORE:
0.7364402064427006


In [218]:
fe = best_model.fit(X_train,y_train).feature_importances_

In [237]:
X_train.shape

(208480, 86)

In [241]:
fe # so there are 69 starting columns + 17 clusters in the end. We see that some clusters bring in tremendous impact.

X.columns[fe[:69]>0.01]

Index(['geo_level_1_id', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_cement_mortar_brick', 'foundation_type_i',
       'foundation_type_r', 'ground_floor_type_v'],
      dtype='object')

In [245]:
np.array(enc_cols)[fe[69:] > 0.01] #important clusters

array(['cluster_4', 'cluster_11', 'cluster_12', 'cluster_14',
       'cluster_15', 'cluster_16'], dtype='<U10')

### Outputing the result

#### Refitting model on the whole data

In [208]:
estim = XGBClassifier(n_estimators = 100,max_depth = 12)

estim.fit(X[:,3:],y['damage_grade'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

#### Predicting on the X_check data

In [209]:
y_all_pred = estim.predict(X_check[:,3:])

In [211]:
y_all_pred.shape

(86868,)

In [212]:
building_ids = X_check['building_id']

In [213]:
def create_submission(y_all_pred,building_ids):

    building_ids = np.array(X_check['building_id'])
    preds = y_all_pred
    sub = pd.DataFrame()
    sub['building_id'] = building_ids
    sub['damage_grade'] = y_all_pred
    sub.set_index('building_id',inplace = True)
    print(sub.shape)
    sub.to_csv('submissions/sub3.csv')

In [214]:
create_submission(y_all_pred,building_ids)

(86868, 1)
